<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [3]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [4]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,6392072535155213407,323758
1,8387817883864991792,331629
2,328012383083104805,359891
3,9183605629983195042,329975
4,8391424625589759186,364497


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [5]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13375 examples in the train dataset and 3273 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [6]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,5.436599,True,12,1,39.0,1451354159195218418
1,6.499227,True,13,1,34.0,524531196325542205
2,6.686620,False,13,1,38.0,2013084202883420573
3,7.577288,True,13,1,40.0,1148502204931914436
4,7.500126,True,13,1,37.0,1148502204931914436


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [7]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13368.000000,13375.000000,13375.000000,13265.000000,1.337500e+04
mean,7.215937,27.380710,1.036037,38.597361,4.425355e+18
std,1.338819,6.188586,0.196163,2.600955,2.795168e+18
min,0.500449,12.000000,1.000000,17.000000,1.244589e+17
25%,6.563162,22.000000,1.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.054038,32.000000,1.000000,40.000000,7.170970e+18
max,18.000744,50.000000,4.000000,47.000000,9.183606e+18


In [8]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [9]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,5.436599,True,12,Single(1),39.0,1451354159195218418
1,6.499227,True,13,Single(1),34.0,524531196325542205
2,6.686620,False,13,Single(1),38.0,2013084202883420573
3,7.577288,True,13,Single(1),40.0,1148502204931914436
4,7.500126,True,13,Single(1),37.0,1148502204931914436


In [10]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13370,7.561856,Unknown,47,Single(1),38.0,7895641210289919997
13371,6.179557,Unknown,49,Multiple(2+),36.0,9183605629983195042
13372,4.034459,Unknown,49,Multiple(2+),37.0,9068386407968572094
13373,7.936641,Unknown,50,Single(1),44.0,4329667052416032880
13374,4.254922,Unknown,50,Multiple(2+),32.0,124458947937444850


In [11]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26518.000000,26518.000000,26518.000000,2.651800e+04
mean,7.215981,27.383966,38.600724,4.424274e+18
std,1.339594,6.187881,2.590042,2.794480e+18
min,0.500449,12.000000,17.000000,1.244589e+17
25%,6.563162,22.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.057896,32.000000,40.000000,7.170970e+18
max,18.000744,50.000000,47.000000,9.183606e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [12]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [13]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6508 eval.csv
  26518 train.csv
  33026 total
==> eval.csv <==
6.0009827716399995,False,14,Single(1),40.0,4740473290291881219
7.3744626639,False,17,Single(1),42.0,4740473290291881219
7.3744626639,True,17,Single(1),36.0,4740473290291881219
6.1244416383599996,True,17,Single(1),41.0,4740473290291881219
6.93794738514,True,18,Single(1),40.0,4740473290291881219
6.8122838958,True,18,Single(1),42.0,4740473290291881219
8.437090766739999,False,18,Single(1),34.0,4740473290291881219
5.43659938092,False,18,Single(1),28.0,4740473290291881219
7.81318256528,True,19,Single(1),38.0,4740473290291881219
7.1870697412,False,19,Single(1),40.0,4740473290291881219

==> train.csv <==
5.43659938092,True,12,Single(1),39.0,1451354159195218418
6.4992274837599995,True,13,Single(1),34.0,524531196325542205
6.686620406459999,False,13,Single(1),38.0,2013084202883420573
7.57728794494,True,13,Single(1),40.0,1148502204931914436
7.50012615324,True,13,Single(1),37.0,1148502204931914436
5.74965579296,True,13,Single(1),35.0

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License